In [ ]:
!pip install --upgrade gupload
from pydrive.auth import GoogleAuth
from google.colab import auth
import os
import requests
import pandas as pd
import numpy as np
import time
import math
import datetime
!pip install nba_api

from nba_api.stats.static import players
from nba_api.stats.static import teams 
from nba_api.stats.endpoints import boxscoreplayertrackv2
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
import time
from timeit import default_timer as timer
from bs4 import BeautifulSoup
from google.colab import drive
auth.authenticate_user()



drive.mount('/content/gdrive')
player_injury_history = pd.read_csv('/content/gdrive/My Drive/FINAL_player_injury_history.csv')
df = pd.read_csv('/content/gdrive/My Drive/FINAL_injuries_post2013.csv')
player_injury_history=player_injury_history.reset_index(drop=False)
# print(player_injury_history)

# print(player_injury_history['Player name'])
columns_to_rename = player_injury_history.columns
print(columns_to_rename)
player_injury_history.columns = ['index', 'Player name', 'foot', 'knee', 'lower leg', 'upper leg', 'back', 'arm', 'hand', 'upper body', 'core and hip', 'head and neck', 'foot_count', 'knee_count', 'lower leg_count', 'upper leg_count', 'back_count', 'arm_count', 'hand_count', 'upper body_count', 'core and hip_count', 'head and neck_count']
# print(player_injury_history)
rng = player_injury_history.shape[0]
player_injury_history_dict = {}
vars = ['foot', 'knee', 'lower leg', 'upper leg', 'back', 'arm', 'hand', 'upper body', 'core and hip', 'head and neck', 'foot_count', 'knee_count', 'lower leg_count', 'upper leg_count', 'back_count', 'arm_count', 'hand_count', 'upper body_count', 'core and hip_count', 'head and neck_count']
for i in range(rng):
  row = player_injury_history.iloc[i,:]
  player_name = row['Player name']
  row2add = {}
  for part in vars:
    row2add[part] = row[part]
  player_injury_history_dict[player_name] = row2add
print(player_injury_history_dict)
print('this one')

#adding columns to the 2013-2019 injuries dataframe that show us the player's previous injury history
#the injury history continuously changes with each injury the player incurs - 
for part in vars:
  df[part] = 0
foot_loc = df.columns.get_loc('foot')
knee_loc = df.columns.get_loc('knee')
lower_leg_loc = df.columns.get_loc('lower leg')
upper_leg_loc = df.columns.get_loc('upper leg')
back_loc = df.columns.get_loc('back')
arm_loc = df.columns.get_loc('arm')
hand_loc = df.columns.get_loc('hand')
upper_body_loc = df.columns.get_loc('upper body')
coreandhip_loc = df.columns.get_loc('core and hip')
head_and_nec_loc = df.columns.get_loc('head and neck')
foot_count_loc = df.columns.get_loc('foot_count')
knee_count_loc = df.columns.get_loc('knee_count')
lower_leg_count_loc = df.columns.get_loc('lower leg_count')
upper_leg_count_loc = df.columns.get_loc('upper leg_count')
back_count_loc = df.columns.get_loc('back_count')
arm_count_loc = df.columns.get_loc('arm_count')
hand_count_loc = df.columns.get_loc('hand_count')
upper_body_count_loc = df.columns.get_loc('upper body_count')
coreandhip_count_loc = df.columns.get_loc('core and hip_count')
head_and_nec_count_loc = df.columns.get_loc('head and neck_count')

vars_loc = [foot_loc,knee_loc,lower_leg_loc,upper_leg_loc,back_loc,arm_loc,hand_loc,upper_body_loc,coreandhip_loc,head_and_nec_loc, foot_count_loc,knee_count_loc,lower_leg_count_loc,upper_leg_count_loc,back_count_loc,arm_count_loc,hand_count_loc,upper_body_count_loc,coreandhip_count_loc,head_and_nec_count_loc]
vars_dict = {}
for i in range(len(vars)):
  name = vars[i]
  loc = vars_loc[i]
  vars_dict[name] = loc
rng = df.shape[0]
for i in range(rng):
  row = df.iloc[i,:]
  player_name = row['Affected Player']
  injury_part = row['Injury Part']
  injury_duration = row['Injury Duration']
  if player_name in player_injury_history_dict:
    for z in range(len(vars)):
      inj_part_name = vars[z]
      inj_part_loc = vars_dict[inj_part_name]
      prev_duration = player_injury_history_dict[player_name][inj_part_name]
      df.iat[i,inj_part_loc]=prev_duration
      if inj_part_name == injury_part:
        player_injury_history_dict[player_name][injury_part] += injury_duration
        player_injury_history_dict[player_name][injury_part+'_count'] += 1
      else:
        continue
  else:
    row2add = {}
    for i in range(len(vars)):
      row2add[vars[i]] = row[vars[i]]
    player_injury_history_dict[player_name] = row2add
    player_injury_history_dict[player_name][injury_part] += injury_duration
    player_injury_history_dict[player_name][injury_part+'_count'] += 1
df['Total Duration'] = 0
df['Total num. of inj'] = 0
total_dur_loc = df.columns.get_loc('Total Duration')
total_nofinj_loc = df.columns.get_loc('Total num. of inj')
for i in range(rng):
  row = df.iloc[i,:]
  total_duration = 0
  total_nofinj = 0
  for part in vars[:10]:
    total_duration += row[part]
  for c in vars[10:]:
    total_nofinj += row[c]
  df.iat[i,total_dur_loc] = total_duration
  df.iat[i,total_nofinj_loc] = total_nofinj

start_season = 2013
end_year_season = 14
seasons_yr_start =[]
seasons_yr_end =[]
for i in range(6):
  seasons_yr_start.append(str(start_season+i))
  seasons_yr_end.append(str(end_year_season+i))
season_names=[]
for r in range(len(seasons_yr_start)):
  season_names.append(seasons_yr_start[r]+'-' + seasons_yr_end[r])
print(season_names)
season_start_dates = ['2013-10-29','2014-10-28','2015-10-27','2016-10-25','2017-10-17','2018-10-16']
szn_end_dates = ['2014-04-17','2015-04-16','2016-04-14','2017-04-13','2018-04-12','2019-04-11']
season_start_dates_dict = {}
season_end_dates_dict = {}
for i in range(len(season_names)):
  season_start_dates_dict[season_names[i]] = season_start_dates[i]
  season_end_dates_dict[season_names[i]] = szn_end_dates[i]

rng = df.shape[0]
df['days since start'] = 0
df['Date']=pd.to_datetime(df['Date'])
days_since_loc = df.columns.get_loc('days since start')
for i in range(rng):
  row = df.iloc[i,:]
  season = row['Season']
  date = row['Date']
  strtregdate = datetime.datetime.strptime(season_start_dates_dict[season], '%Y-%m-%d')
  if date <= strtregdate:
    df.iat[i,days_since_loc]=0
  else:
    dist = (date - strtregdate).days
    df.iat[i,days_since_loc] = dist
injuries_df = df.copy()
df=df.reset_index(drop=False)


#dropping injuries that occur 2 weeks or before the start of the season
#due to how it would skew the AWCN data

rng = df.shape[0]
to_drop = []
for i in range(rng):
  row = df.iloc[i,:]
  season = row['Season']
  date = row['Date']
  days_since_start = row['days since start']
  if days_since_start<14:
    to_drop.append(i)
df = df.drop(labels=to_drop,axis=0)
df=df.reset_index(drop=False)
df['Season Type'] = 'Regular Season'
szn_type_loc = df.columns.get_loc('Season Type')
rng = df.shape[0]
for i in range(rng):
  row = df.iloc[i,:]
  season = row['Season']
  date = row['Date']
  endregdate = datetime.datetime.strptime(season_end_dates_dict[season], '%Y-%m-%d')
  if date <= endregdate:
    df.iat[i,szn_type_loc] = 'Regular Season'
  else:
    df.iat[i,szn_type_loc] = 'Playoffs'


for i in range(len(szn_end_dates)):
  szn_end_dates[i] = datetime.datetime.strptime(szn_end_dates[i], '%Y-%m-%d')
  szn_end_dates[i] = datetime.datetime.strptime(datetime.datetime.strftime(szn_end_dates[i], '%m/%d/%Y'),'%m/%d/%Y')
  print ((szn_end_dates[i]))
  print(type(szn_end_dates[i]))
from datetime import timedelta
import time
playoff_start_dates = {}
for i in range(len(szn_end_dates)):
  playoff_starts = (szn_end_dates[i] + timedelta(days = 2))
  DT_version = datetime.datetime.strptime(datetime.datetime.strftime(playoff_starts,'%m/%d/%Y'),'%m/%d/%Y')
  str_version = datetime.datetime.strftime(playoff_starts,'%m/%d/%Y')
  lst = [DT_version,str_version]
  playoff_start_dates[season_names[i]] = lst
print(playoff_start_dates)
# .dt.strftime('%m/%d/%Y')

df['Date']=pd.to_datetime(df['Date'])
df['Date_adjusted'] = pd.to_datetime(df['Date'].dt.strftime('%m/%d/%Y'))
df['7_day_back'] = (df['Date'] - pd.to_timedelta(7, unit='D'))
df['28_day_back'] = 0
day_28_loc = df.columns.get_loc('28_day_back')
rng = df.shape[0]
for i in range(rng):
  row = df.iloc[i,:]
  if row['days since start']<28:
    df.iat[i,day_28_loc] = (row['Date'] - pd.to_timedelta(row['days since start'], unit='D'))
  else:
    df.iat[i,day_28_loc] = (row['Date'] - pd.to_timedelta(28, unit='D'))
df['28_day_back'] = pd.to_datetime(df['28_day_back'])
df['28_day_back'] = df['28_day_back'].dt.strftime('%m/%d/%Y')
df['7_day_back'] = pd.to_datetime(df['7_day_back'])
df['7_day_back'] = df['7_day_back'].dt.strftime('%m/%d/%Y')
rng = df.shape[0]
def add_playoff_start_date(row):
  szn = row['Season']
  return playoff_start_dates[szn][1]
df['playoff start date']= df.apply(lambda ro:add_playoff_start_date(ro), axis=1)
df['days_since_start_of_playoffs'] = 0
playoffs_difference_loc = df.columns.get_loc('days_since_start_of_playoffs')
for i in range(rng):
  row = df.iloc[i,:]
  szn_type = row['Season Type']
  szn = row['Season']
  POstart = playoff_start_dates[szn][0]
  if szn_type == 'Regular Season':
    df.iat[i,playoffs_difference_loc] = 0
  else:
    df.iat[i,playoffs_difference_loc] = (row['Date'] - POstart).days
df['Date_adjusted']=df['Date_adjusted'].dt.strftime('%m/%d/%Y')

teams = teams.get_teams()
player_dict = players.get_players()
player_id_dict = {}
for key in player_dict:
    player_id_dict[key['full_name']] = key['id']
team_id_dict = {}
for team in teams:
    team_id_dict[team['nickname']] = team['id']
rcl = df.columns.get_loc('Relinquished')
acl = df.columns.get_loc('Affected Player')
player_dict = players.get_players()
lst_of_player_names = []
for key in player_dict:
    lst_of_player_names.append((key['full_name']))
rng = df.shape[0]
adjust_to_this = {'C.J. Miles':'CJ Miles', 'J.J. Redick':'JJ Redick', 'J.J. Hickson':"JJ Hickson",'C.J. McCollum':"CJ McCollum",'P.J. Hairston':'PJ Hairston','Dennis Schröder':'Dennis Schroder','J.R. Smith':"JR Smith",'T.J. Leaf': 'TJ Leaf', 'Ogugua Anunoby / O.G. Anunoby':'OG Anunoby', 'James Ennis':'James Ennis III','Marcus Morris':"Marcus Morris Sr.", 'Harry Giles':'Harry Giles III', 'Danuel House':'Danuel House Jr.'}
adjust_to_this_team = {'Bobcats':'Hornets', 'Blazers':'Trail Blazers'}
names_in_df = []
for i in range(rng):
    row = df.iloc[i,:]
    player_name_lower = row.loc['Affected Player'].lower()
    player_name = row.loc['Affected Player']
    relinquished_name = row.loc['Relinquished']
    acquired_name = row.loc['Acquired']
    if player_name in adjust_to_this:
        df.iat[i,rcl] = adjust_to_this[player_name]
        df.iat[i,acl] = adjust_to_this[player_name]
team_loc = df.columns.get_loc('Team')
for i in range(rng):
    row = df.iloc[i,:]
    team_name = row['Team']
    if team_name in adjust_to_this_team:
        df.iat[i,team_loc] = adjust_to_this_team[team_name]
    else:
        continue
            
def get_player_ids(row):
    player_name = row['Affected Player']
    if player_name in player_id_dict:
        return player_id_dict[player_name]
    else:
        return 'False'
def get_team_ids(row):
    team_name = row['Team']
    if team_name in team_id_dict:
        return team_id_dict[team_name]
    else:
        return 'False'

df['player_id']=df.apply(lambda x:get_player_ids(x), axis=1)
df['Team_id']=df.apply(lambda x:get_team_ids(x), axis=1)


df.to_csv('Final_2013_Version_with_counts.csv')
!gupload --to '12W7wtP_PL3oV5osKCwxM2cQWxzu1yETv' REVISED_Final_2013_Version.csv
print('Finished')
print('Done latest iteration')




Requirement already up-to-date: gupload in /usr/local/lib/python3.7/dist-packages (1.1.0)
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Index(['index', 'Unnamed: 0', 'foot', 'knee', 'upper leg', 'lower leg', 'back',
       'arm', 'hand', 'upper body', 'core and hip', 'head and neck',
       'foot_count', 'knee_count', 'upper leg_count', 'lower leg_count',
       'back_count', 'arm_count', 'hand_count', 'upper body_count',
       'core and hip_count', 'head and neck_count'],
      dtype='object')
{'Kenny Battle': {'foot': 0, 'knee': 0, 'lower leg': 0, 'upper leg': 0, 'back': 0, 'arm': 0, 'hand': 14, 'upper body': 0, 'core and hip': 0, 'head and neck': 0, 'foot_count': 0, 'knee_count': 0, 'lower leg_count': 0, 'upper leg_count': 0, 'back_count': 0, 'arm_count': 0, 'hand_count': 1, 'upper body_count': 0, 'core and hip_count': 0, 'head and neck_count': 0}, 'Marcus Webb': {'foot': 0, 'knee': 0, 'lower leg':

In [ ]:
vars = ['foot', 'knee', 'lower leg', 'upper leg', 'back', 'arm', 'hand', 'upper body', 'core and hip', 'head and neck','foot_count', 'knee_count', 'lower leg_count', 'upper leg_count', 'back_count', 'arm_count', 'hand_count', 'upper body_count', 'core and hip_count', 'head and neck_count']
print(len(vars[10:]))

10
